This is a test of the translation of the DSS algorithm from Matlab to Python. The Matlab source is taken from `NoiseTools` package that can be found [here](http://audition.ens.fr/adc/NoiseTools/). 

The test consists of running an example from the `NoiseTools` in parallel in Matlab and Python and matching the results.

# Setup

## Imports

In [1]:
from pathlib import Path
import inspect

In [2]:
# Code highlightintg
from pygments import highlight
from pygments.lexers import MatlabLexer, PythonLexer
from pygments.formatters import HtmlFormatter

In [3]:
from IPython.core.display import HTML

In [4]:
# This is a package that allows us to inerface with a Matlab process
from pymatbridge import Matlab

C:\Users\ekalenkovich\Miniconda3\envs\ding_vs_frank_and_yang\lib\site-packages\IPython\nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


In [5]:
import numpy as np
import scipy
from matplotlib import pyplot as plt

## Set up autoreloading of the Python code

In [6]:
%load_ext autoreload
%autoreload 1  # autoreload only the source loaded with %aimport
%aimport dss

## Paths

Change this to wherever you unpacked `NoiseTools` to.

In [7]:
noise_tools_dir = Path('NoiseTools')

In [8]:
noise_tools_examples_dir = noise_tools_dir / 'EXAMPLES' / 'a few old example scripts'
example_1 = noise_tools_examples_dir / 'example1.m'

## Printing code

In [9]:
def print_highlighted_code(code, lexer):
    display(HTML(highlight(code, lexer, HtmlFormatter(full=True, linenos='inline'))))
    
def print_matlab_code(code):
    print_highlighted_code(code, MatlabLexer())
    
def print_python_code(code):
    print_highlighted_code(code, PythonLexer())

def print_matlab_script(path):
    with open(path, 'r') as f:
        code = f.read()
    print_matlab_code(code)
    
def print_python_function(fun):
    code = inspect.getsource(fun)
    print_python_code(code)

## Start Matlab

In [10]:
matlab = Matlab()
matlab.start()

Starting MATLAB on ZMQ socket tcp://127.0.0.1:51661
Send 'exit' command to kill the server
.......MATLAB started and connected!


## Add `NoiseTools` to the Matlab's path

In [11]:
matlab.run_code('addpath(\'{}\')'.format(noise_tools_dir))

{'content': {'stdout': '',
  'figures': [],
  'datadir': 'C:\\Users\\EKALEN~1\\AppData\\Local\\Temp\\MatlabData\\'},
 'result': [],
 'success': True}

# Simulate data

Let's look at the example 1 code:

In [12]:
print_matlab_script(example_1)

Let's create synthetic data in Matlab and transfer it here.

In [13]:
example_1_code = open(example_1, 'r').readlines()
synthethize_data_code = ''.join(example_1_code[9:21])
print_matlab_code(synthethize_data_code)

In [14]:
matlab.run_code(synthethize_data_code)
data = matlab.get_variable('data')
print(data.shape)

(300, 30, 100)


That is 300 time points, 30 channels and 100 trials.

# Calculate covariances

## Inspect the `NoiseTools` code

Here is how the covariance matrices are calculated:

In [15]:
covariances_code = ''.join(example_1_code[22:24])
print_matlab_code(covariances_code)

Let's see what `nt_cov` does.

In [16]:
nt_cov = noise_tools_dir / 'nt_cov.m'
print_matlab_script(nt_cov)

The example code only uses the first argument `x` and only the first output `c`. Also, both `data` and `mean(data,3)` are single matrices, not arrays of matrices. Therefore, checks on line 33 (`isempty(w)`) and on line 35 (`is_numberic(x)`) both avaluate to `true`. So, the only piece of code relevant to us is this:

In [17]:
nt_cov_code = open(nt_cov, 'r').readlines()
print_matlab_code(''.join(nt_cov_code[35:42]))

Let's see what `nt_multishift` does. First, we need to figure out what `shifts` is since we didn't supply this parameter. Here are the relevant lines:

In [18]:
print_matlab_code(''.join(nt_cov_code[26:29]))

So, shifts is just `0`. Let's see what `nt_multishift` does in this case.

In [19]:
nt_multishift_path = noise_tools_dir / 'nt_multishift.m'
print_matlab_script(nt_multishift_path)

And here is the relevant part:

In [20]:
nt_multishift_code = open(nt_multishift_path, 'r').readlines()
print_matlab_code(''.join(nt_multishift_code[28:32]))

So, with `shifts == 0` function `nt_multishift` does not change the input. Let's get back to the `nt_cov` code then.

In [21]:
nt_cov_code = open(nt_cov, 'r').readlines()
print_matlab_code(''.join(nt_cov_code[35:42]))

Since `data` is shaped as `nsamples`-by-`nchans`-by-`ntrials` then so is `x`.

For each `k`:
- `xx` is the data from one trial,
- `xx'*xx` is uncentered covariance of channels in time during a single trial.

Then `c` is the sum of per-trial ucnentered covariances.

## Run the Matlab code

In [22]:
print_matlab_code(covariances_code)

In [23]:
matlab.run_code(covariances_code)
c0 = matlab.get_variable('c0')
c1 = matlab.get_variable('c1')

## Run the Python code

In [24]:
print_python_function(dss.calculate_covariances)

In [25]:
R0, R1 = dss.calculate_covariances(data, uncentered=True, unnormalized=True)

## Match

In [26]:
np.allclose(R0, c0)

True

In [27]:
np.allclose(R1, c1)

True

# PCA

The DSS algorithm includes two PCA rotations. Let's check that we do it in the same manner.

## Take a look at the `NoiseTools` code

In [28]:
pca_path = noise_tools_dir / 'nt_pcarot.m'
print_matlab_script(pca_path)

We won't be working with sparse matrices so the Python code won't have the `N` parameter.

## Take a look at the Python code

In [29]:
print_python_function(dss.covariance_pca)

## Run the code

### Matlab

In [30]:
matlab.run_code('[E, D] = nt_pcarot(c0);')
E = matlab.get_variable('E')
D = matlab.get_variable('D')

### Python

In [31]:
eigvals, eigvecs = dss.covariance_pca(R0)

## Match

In [32]:
np.allclose(eigvals, D)

True

In [33]:
np.allclose(eigvecs, E)

False

Let's take a look at the topleft corners.

In [34]:
eigvecs[:5, :5]

array([[ 0.12823076, -0.12522033,  0.04200415, -0.2117002 ,  0.13922945],
       [ 0.13930787, -0.09322159, -0.19794139,  0.1359117 , -0.04624764],
       [-0.23881671,  0.01603279,  0.0706665 ,  0.07640401, -0.06440901],
       [-0.04584022,  0.04905365,  0.27516218, -0.1532583 ,  0.12172959],
       [-0.31901507,  0.08675316,  0.2753377 , -0.03713511,  0.23694942]])

In [35]:
E[:5, :5]

array([[ 0.12823076, -0.12522033,  0.04200415, -0.2117002 , -0.13922945],
       [ 0.13930787, -0.09322159, -0.19794139,  0.1359117 ,  0.04624764],
       [-0.23881671,  0.01603279,  0.0706665 ,  0.07640401,  0.06440901],
       [-0.04584022,  0.04905365,  0.27516218, -0.1532583 , -0.12172959],
       [-0.31901507,  0.08675316,  0.2753377 , -0.03713511, -0.23694942]])

The first problem is some vectors differ in sign. Let's coerce the first row of both matrices to be positive.

In [36]:
eigvecs *= np.sign(eigvecs[np.newaxis, 0])
E = E * np.sign(E[np.newaxis, 0])

In [37]:
np.allclose(eigvecs, E)

False

The second problem is that eigenvectors corresponding to the very small eigenvalues may differ a lot.

In [38]:
threshold = 1e-9
abs_threshold = max(eigvals) * threshold
np.allclose(eigvecs[:, eigvals > abs_threshold], E[:, D.squeeze() > abs_threshold])

True

So, the results are equivalent. Let's check that both functions produce the same output if we ask them to remove eigenvalues below a given threshold.

In [39]:
matlab.set_variable(value=threshold, varname='threshold')
matlab.run_code('[E, D] = nt_pcarot(c0, [], threshold);')
E = matlab.get_variable('E')
D = matlab.get_variable('D')
eigvals, eigvecs = dss.covariance_pca(R0, threshold=threshold)

In [40]:
np.allclose(eigvals, D)

True

In [41]:
eigvecs *= np.sign(eigvecs[np.newaxis, 0])
E = E * np.sign(E[np.newaxis, 0])
np.allclose(eigvecs, E)

True